In [20]:
import qrcode
from PIL import Image, ImageDraw, ImageFont
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
import pandas as pd
import os
from tkinter import Tk, messagebox
from tkinter.filedialog import askopenfilename, askdirectory

In [21]:
# Initialize Tkinter and show the root window
root = Tk()
root.withdraw()  # Hide the main Tkinter window

# Open a file dialog to select the CSV file
csv_file_path = askopenfilename(title="Select the students CSV file", filetypes=[("CSV files", "*.csv")])
if not csv_file_path:
    messagebox.showerror("Error", "No CSV file selected. Exiting.")
    root.destroy()
    exit()

# Load student data from the selected CSV file
students = pd.read_csv(csv_file_path)  # Ensure your CSV has columns 'StudentID', 'Name', 'Class'

# Open a directory dialog to select the directory for QR codes
qr_codes_dir = askdirectory(title="Select the directory to store QR codes")
if not qr_codes_dir:
    messagebox.showerror("Error", "No directory selected. Exiting.")
    root.destroy()
    exit()

os.makedirs(qr_codes_dir, exist_ok=True)

In [22]:
# Generate QR codes for each student and save as PNG with custom design
qr_codes = []
for index, row in students.iterrows():
    student_id = row['StudentID']
    name = row['Name']
    class_name = row['Class']
    qr_data = f'{class_name}:{student_id}'
    
    # Generate QR code
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=2,
    )
    qr.add_data(qr_data)
    qr.make(fit=True)
    img = qr.make_image(fill='black', back_color='white').convert('RGB')
    
    # Create custom design
    img = img.resize((200, 200), Image.LANCZOS)
    custom_img = Image.new('RGB', (200, 250), 'white')
    draw = ImageDraw.Draw(custom_img)
    
    # Load a font
    font = ImageFont.load_default()  # You can replace this with a TTF font for better styling
    
    # Draw the student ID above the QR code
    bbox = draw.textbbox((0, 0), student_id, font=font)
    text_width = bbox[2] - bbox[0]
    draw.text(((200 - text_width) / 2, 10), student_id, fill='black', font=font)
    
    # Paste the QR code below the text
    custom_img.paste(img, (0, 40))
    
    # Save as PNG
    img_path = os.path.join(qr_codes_dir, f'{student_id}.png')
    custom_img.save(img_path)
    qr_codes.append(img_path)

In [23]:
# Function to create PDF with arranged QR codes
def create_pdf(qr_codes, output_filename):
    c = canvas.Canvas(output_filename, pagesize=A4)
    width, height = A4
    qr_width = 180  # Width of each QR code with design
    qr_height = 250  # Height of each QR code with design
    margin = 5  # Margin around QR codes
    columns = 3  # Number of columns of QR codes per page
    rows = 3  # Number of rows of QR codes per page

    x_offset = margin
    y_offset = height - margin - qr_height

    for i, qr_code in enumerate(qr_codes):
        if i > 0 and i % (columns * rows) == 0:
            c.showPage()
            x_offset = margin
            y_offset = height - margin - qr_height

        c.drawImage(qr_code, x_offset, y_offset, qr_width, qr_height)

        x_offset += qr_width + margin
        if (i + 1) % columns == 0:
            x_offset = margin
            y_offset -= qr_height + margin

    c.save()
    print(f"PDF created at: {output_filename}")

output_pdf_path = os.path.join(qr_codes_dir, 'qr_codes.pdf')
create_pdf(qr_codes, output_pdf_path)

PDF created at: /Users/jbs939/Documents/StudioRepo/Attenderly/Outputs/Student QR coders/qr_codes.pdf


In [24]:
# Properly destroy the Tkinter root window
root.destroy()

# Print completion message to the console instead of using messagebox
print(f"PDF created at: {output_pdf_path}")

PDF created at: /Users/jbs939/Documents/StudioRepo/Attenderly/Outputs/Student QR coders/qr_codes.pdf


: 